<a href="https://colab.research.google.com/github/Fet-elena/proba/blob/main/HW_3_ML_Fetisova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [ ]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [ ]:
y = data['MEDV']
X = data.drop(columns=['MEDV'])
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2, random_state = RANDOM_STATE)



In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((404, 13), (102, 13), (404,), (102,))

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train) # обучим нашу модель
y_pred = model.predict(X_test) # предскажем значения y
print('R2 Linear regression:', r2_score(y_test, y_pred))

#r2_lr = 0
#r2_ridge = 0
#r2_lasso = 0
#results_regression.loc[0] = ['LR', 'task2', r2_lr]
#results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
#results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

R2 Linear regression: 0.6684825753971617


**Ridge**

In [ ]:
model_ridge = Ridge()
model_ridge.fit(X_train, y_train)
y_pred_ridge = model_ridge.predict(X_test) # предскажем значения y
print('R2 Ridge:', r2_score(y_test, y_pred_ridge))


R2 Ridge: 0.6659608075261694


**Lasso**

In [ ]:
model_lasso = Lasso()
model_lasso.fit(X_train, y_train)
y_pred_lasso = model_lasso.predict(X_test) # предскажем значения y
print('R2 Lasso:', r2_score(y_test, y_pred_lasso))

R2 Lasso: 0.6668687223368213


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [40]:
alpha_values = {'alpha': np.logspace(-5, 5, 10)}
ridge = GridSearchCV(Ridge(),
                     alpha_values,
                     scoring = 'neg_mean_squared_error',
                     cv = 10)
print('The best value of alpha is:', ridge.fit(X_train,y_train).best_params_)
lasso = GridSearchCV(Lasso(),
                     alpha_values,
                     scoring = 'neg_mean_squared_error',
                     cv = 10)
print(ridge.fit(X_train,y_train).best_params_,lasso.fit(X_train,y_train).best_params_)

ridge_GSCV_best = ridge.best_estimator_
ridge_GSCV_r2 = r2_score(y_test, ridge_GSCV_best.predict(X_test))

lasso_GSCV_best = lasso.best_estimator_
lasso_GSCV_r2 = r2_score(y_test, lasso_GSCV_best.predict(X_test))
print('R2 Ridge_GridSearchCV:', ridge_GSCV_r2)
print('R2 Lasso_GridSearchCV:', lasso_GSCV_r2)

RidCV = RidgeCV(alphas = np.logspace(-5, 5, 10))
RidCV.fit(X_train, y_train)
print('alpha=', RidCV.alpha_)

RidCV_best = Ridge(alpha=RidCV.alpha_)
RidCV_best.fit(X_train, y_train)

Ridge_RidCV_r2 = r2_score(y_test, RidCV_best.predict(X_test))
print('R2 Ridge_RidgeCV:', Ridge_RidCV_r2)

LassCV = LassoCV(alphas = np.logspace(-5, 5, 10))
LassCV.fit(X_train, y_train)
print('alpha=', LassCV.alpha_)

Lasso_LassCV_r2 = r2_score(y_test, LassCV.predict(X_test))
print('R2 Lasso_LassoCV:', Lasso_LassCV_r2)


The best value of alpha is: {'alpha': 1e-05}
{'alpha': 1e-05} {'alpha': 1e-05}
R2 Ridge_GridSearchCV: 0.6684825680074258
R2 Lasso_GridSearchCV: 0.6684829595885677
alpha= 0.021544346900318846
R2 Ridge_RidgeCV: 0.6684636466375218
alpha= 1e-05
R2 Lasso_LassoCV: 0.6684829595885677


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [ ]:
Linear_StScal = Pipeline(steps = [("scaler", StandardScaler()), ("regression", LinearRegression())])
Linear_StScal.fit(X_train, y_train)

Linear_StScal_r2 = r2_score(y_test, Linear_StScal.predict(X_test))
print('R2_Linear_regression_StandardScaler:', Linear_StScal_r2)

Ridge_StScal = Pipeline(steps = [("scaler", StandardScaler()), ("regression", Ridge())])
Ridge_StScal.fit(X_train, y_train)

Ridge_StScal_r2 = r2_score(y_test, Ridge_StScal.predict(X_test))
print('R2_Ridge_StandardScaler:', Ridge_StScal_r2)

Lasso_StScal = Pipeline(steps = [("scaler", StandardScaler()), ("regression", Lasso())])
Lasso_StScal.fit(X_train, y_train)

Lasso_StScal_r2 = r2_score(y_test, Lasso_StScal.predict(X_test))
print('R2_Lasso_StandardScaler:', Lasso_StScal_r2)

Linear_MinMaxScal = Pipeline(steps = [("scaler", MinMaxScaler()), ("regression", LinearRegression())])
Linear_MinMaxScal.fit(X_train, y_train)

Linear_MinMaxScal_r2 = r2_score(y_test, Linear_MinMaxScal.predict(X_test))
print('R2_Linear_MinMaxScaler:', Linear_MinMaxScal_r2)

Ridge_MinMaxScal = Pipeline(steps = [("scaler", MinMaxScaler()), ("regression", Ridge())])
Ridge_MinMaxScal.fit(X_train, y_train)

Ridge_MinMaxScal_r2 = r2_score(y_test, Ridge_MinMaxScal.predict(X_test))
print('R2_Ridge_MinMaxScaler:', Ridge_MinMaxScal_r2)

Lasso_MinMaxScal = Pipeline(steps = [("scaler", MinMaxScaler()), ("regression", Lasso())])
Lasso_MinMaxScal.fit(X_train, y_train)

Lasso_MinMaxScal_r2 = r2_score(y_test, Lasso_MinMaxScal.predict(X_test))
print('R2_Lasso_MinMaxScaler:', Lasso_MinMaxScal_r2)

R2_Linear_regression_StandardScaler: 0.6684825753971608
R2_Ridge_StandardScaler: 0.668190107677443
R2_Lasso_StandardScaler: 0.624044752347846
R2_Linear_MinMaxScaler: 0.6684825753971607
R2_Ridge_MinMaxScaler: 0.6762207658974595
R2_Lasso_MinMaxScaler: 0.2573921442545195


Коэффициент детерминации у линейной регрессии после масштабирования не изменился.
Коэффициент детерминации у  Ridge  после масштабирования  MinMaxScaler  улучшился.
Коэффициент детерминации у  Lasso  после масштабирования ухудшился.




5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [36]:
alpha_values = {'regression__alpha': np.logspace(-5, 5, 10)}

Ridge_StScal = Pipeline(steps = [("scaler", StandardScaler()), ("regression", Ridge())])
Ridge_StScal_GSCV = GridSearchCV(Ridge_StScal, alpha_values)
Ridge_StScal_GSCV.fit(X_train, y_train)
Ridge_StScal_GSCV_r2 = r2_score(y_test, Ridge_StScal_GSCV.predict(X_test))
print('Alpha для Ridge на StandardScaler:', Ridge_StScal_GSCV.best_params_.get('regression__alpha'))
print('R2_Ridge_StandardScaler:', Ridge_StScal_GSCV_r2)

Ridge_MinMaxScal = Pipeline(steps = [("scaler", MinMaxScaler()), ("regression", Ridge())])
Ridge_MinMaxScal_GSCV = GridSearchCV(Ridge_MinMaxScal, alpha_values)
Ridge_MinMaxScal_GSCV.fit(X_train, y_train)
Ridge_MinMaxScal_GSCV_r2 = r2_score(y_test, Ridge_MinMaxScal_GSCV.predict(X_test))
print('Alpha для Ridge на MinMaxScaler:', Ridge_MinMaxScal_GSCV.best_params_.get('regression__alpha'))
print('R2_Ridge_MinMaxScaler:', Ridge_MinMaxScal_GSCV_r2)

Lasso_StScal = Pipeline(steps = [("scaler", StandardScaler()), ("regression", Lasso())])
Lasso_StScal_GSCV = GridSearchCV(Lasso_StScal, alpha_values)
Lasso_StScal_GSCV.fit(X_train, y_train)
Lasso_StScal_GSCV_r2 = r2_score(y_test, Lasso_StScal_GSCV.predict(X_test))
print('Alpha для Lasso на StandardScaler:', Lasso_StScal_GSCV.best_params_.get('regression__alpha'))
print('R2_Lasso_StandardScaler:', Lasso_StScal_GSCV_r2)

Lasso_MinMaxScal = Pipeline(steps = [("scaler", MinMaxScaler()), ("regression", Lasso())])
Lasso_MinMaxScal_GSCV = GridSearchCV(Lasso_MinMaxScal, alpha_values)
Lasso_MinMaxScal_GSCV.fit(X_train, y_train)
Lasso_MinMaxScal_GSCV_r2 = r2_score(y_test, Lasso_MinMaxScal_GSCV.predict(X_test))
print('Alpha для Lasso на MinMaxScaler:',  Lasso_MinMaxScal_GSCV.best_params_.get('regression__alpha'))
print('R2_Lasso_MinMaxScaler:', Lasso_MinMaxScal_GSCV_r2)



Alpha для Ridge на StandardScaler: 3.593813663804626
R2_Ridge_StandardScaler: 0.6674380728546585
Alpha для Ridge на MinMaxScaler: 0.2782559402207126
R2_Ridge_MinMaxScaler: 0.6716879333432204
Alpha для Lasso на StandardScaler: 1e-05
R2_Lasso_StandardScaler: 0.6684821312777706
Alpha для Lasso на MinMaxScaler: 1e-05
R2_Lasso_MinMaxScaler: 0.6684836564428189


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [ ]:
Ridge_StScal_Polynom = Pipeline(steps = [("scaler", StandardScaler()), ("polynomial", PolynomialFeatures(degree=2)), ("regression", Ridge())])
Ridge_StScal_Polynom.fit(X_train, y_train)
Ridge_StScal_Polynom.predict(X_test)
Ridge_StScal_Polynom_r2 = r2_score(y_test, Ridge_StScal_Polynom.predict(X_test))
print('R2_Ridge_StandardScaler:', Ridge_StScal_Polynom_r2)

Ridge_MinMaxScal_Polynom = Pipeline(steps = [("scaler", MinMaxScaler()), ("polynomial", PolynomialFeatures(degree=2)), ("regression", Ridge())])
Ridge_MinMaxScal_Polynom.fit(X_train, y_train)
Ridge_MinMaxScal_Polynom.predict(X_test)
Ridge_MinMaxScal_Polynom_r2 = r2_score(y_test, Ridge_MinMaxScal_Polynom.predict(X_test))
print('R2_Ridge_MinMaxScaler:', Ridge_MinMaxScal_Polynom_r2)

Lasso_StScal_Polynom = Pipeline(steps = [("scaler", StandardScaler()), ("polynomial", PolynomialFeatures(degree=2)), ("regression", Lasso())])
Lasso_StScal_Polynom.fit(X_train, y_train)
Lasso_StScal_Polynom.predict(X_test)
Lasso_StScal_Polynom_r2 = r2_score(y_test, Lasso_StScal_Polynom.predict(X_test))
print('R2_Lasso_StandardScaler:', Lasso_StScal_Polynom_r2)

Lasso_MinMaxScal_Polynom = Pipeline(steps = [("scaler", MinMaxScaler()), ("polynomial", PolynomialFeatures(degree=2)), ("regression", Lasso())])
Lasso_MinMaxScal_Polynom.fit(X_train, y_train)
Lasso_MinMaxScal_Polynom.predict(X_test)
Lasso_MinMaxScal_Polynom_r2 = r2_score(y_test, Lasso_MinMaxScal_Polynom.predict(X_test))
print('R2_Lasso_MinMaxScaler:', Lasso_MinMaxScal_Polynom_r2)


R2_Ridge_StandardScaler: 0.8171359789657098
R2_Ridge_MinMaxScaler: 0.8298862467502747
R2_Lasso_StandardScaler: 0.7322738282708989
R2_Lasso_MinMaxScaler: 0.2611262741735658


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [ ]:
alpha_values = {'regression__alpha':np.logspace(-5, 5, 10)}

Ridge_StScal_Polynom = Pipeline(steps = [("scaler", StandardScaler()), ("polynomial", PolynomialFeatures(degree=2)), ("regression", Ridge())])
Ridge_StScal_Polynom_GSCV = GridSearchCV(Ridge_StScal_Polynom, alpha_values)
Ridge_StScal_Polynom_GSCV.fit(X_train, y_train)
Ridge_StScal_Polynom_GSCV_r2 = r2_score(y_test, Ridge_StScal_Polynom_GSCV.predict(X_test))
print('R2_Ridge_StandardScaler_PolynomialFeatures_GridSearchCV:', Ridge_StScal_Polynom_GSCV_r2)

Ridge_MinMaxScal_Polynom = Pipeline(steps = [("scaler", MinMaxScaler()), ("polynomial", PolynomialFeatures(degree=2)), ("regression", Ridge())])
Ridge_MinMaxScal_Polynom_GSCV = GridSearchCV(Ridge_MinMaxScal_Polynom, alpha_values)
Ridge_MinMaxScal_Polynom_GSCV.fit(X_train, y_train)
Ridge_MinMaxScal_Polynom_GSCV_r2 = r2_score(y_test, Ridge_MinMaxScal_Polynom_GSCV.predict(X_test))
print('R2_Ridge_MinMaxScaler_PolynomialFeatures_GridSearchCV:', Ridge_MinMaxScal_Polynom_GSCV_r2)

Lasso_StScal_Polynom = Pipeline(steps = [("scaler", StandardScaler()), ("polynomial", PolynomialFeatures(degree=2)), ("regression", Lasso())])
Lasso_StScal_Polynom_GSCV = GridSearchCV(Lasso_StScal_Polynom, alpha_values)
Lasso_StScal_Polynom_GSCV.fit(X_train, y_train)
Lasso_StScal_Polynom_GSCV_r2 = r2_score(y_test, Lasso_StScal_Polynom_GSCV.predict(X_test))
print('R2_Lasso_StandardScaler_PolynomialFeatures_GridSearchCV:', Lasso_StScal_Polynom_GSCV_r2)

Lasso_MinMaxScal_Polynom = Pipeline(steps = [("scaler", MinMaxScaler()), ("polynomial", PolynomialFeatures(degree=2)), ("regression", Lasso())])
Lasso_MinMaxScal_Polynom_GSCV = GridSearchCV(Lasso_MinMaxScal_Polynom, alpha_values)
Lasso_MinMaxScal_Polynom_GSCV.fit(X_train, y_train)
Lasso_MinMaxScal_Polynom_GSCV_r2 = r2_score(y_test, Lasso_MinMaxScal_Polynom_GSCV.predict(X_test))
print('R2_Lasso_MinMaxScaler_PolynomialFeatures_GridSearchCV:', Lasso_MinMaxScal_Polynom_GSCV_r2)



R2_Ridge_StandardScaler_PolynomialFeatures_GridSearchCV: 0.8215015231918026
R2_Ridge_MinMaxScaler_PolynomialFeatures_GridSearchCV: 0.8486007338517558
R2_Lasso_StandardScaler_PolynomialFeatures_GridSearchCV: 0.8171615214862488
R2_Lasso_MinMaxScaler_PolynomialFeatures_GridSearchCV: 0.8497743399242613


8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [ ]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48842 non-null  object
 14  class           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [ ]:
X = data.drop(columns=['class'])
data['class'] = data['class'].replace('<=50K', 1)
data['class'] = data['class'].replace('>50K', 0)
y = data['class']
y.value_counts()


1    37155
0    11687
Name: class, dtype: int64

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [24]:
from sklearn.metrics import accuracy_score, f1_score
y_true=y
y_pred=np.ones_like(y_true)
print(accuracy_score(y_true, y_pred))
print(f1_score(y_true, y_pred))



0.7607182343065395
0.8640999104619929


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [25]:
print(data.isnull().sum())

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64


пропусков нет

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [35]:
columns = data.applymap(np.isreal).all()
True_columns = data.columns[columns].tolist()
False_columns = data.columns[~columns].tolist()
print('Числовые:', True_columns)
print('Категориальные:', False_columns)

Числовые: ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'class']
Категориальные: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [ ]:


### Ваш код ###

f1_LR_del_missings = 0
acc_LR_del_missings = 0
f1_KNN_del_missings = 0
acc_KNN_del_missings = 0
f1_SVM_del_missings = 0
acc_SVM_del_missings = 0
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [ ]:


### Ваш код ###

f1_RF = 0
acc_RF = 0
f1_GB = 0
acc_GB = 0
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:


### Ваш код ###

best_params = {}
print('Параметры лучшей модели:\n', best_params)
f1_best = 0
acc_best = 0
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [ ]:
results_classification